## Purpose
The purpose of this application is to solve relevant classification and regression problems for the prostate dataset for use in the project in 02450 Intro to Machine Learning

Author: Naia Wright

Reviewed by:  

Last modified: 28/10/18, 09:39

#### Change-log


Import of required packages

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
from scipy.linalg import svd

from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import sklearn.linear_model as lm
from matplotlib.pylab import figure, plot, xlabel, ylabel, legend, ylim, show

from matplotlib.pyplot import figure, boxplot, xlabel, ylabel, show
import numpy as np
from scipy.io import loadmat
import sklearn.linear_model as lm
from sklearn import model_selection, tree
from scipy import stats

from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection

from matplotlib.pyplot import figure, plot, subplot, title, xlabel, ylabel, show, clim
from scipy.io import loadmat
import sklearn.linear_model as lm
from sklearn import model_selection
from toolbox_02450 import feature_selector_lr, bmplot
import numpy as np


#### Import of data

First we define a method for importing a spread_sheet using pandas

In [3]:
def DataLoader(path, sheet):
    """
    Method for importing data from a spreadsheet.

    :param path: full path to the spreadsheet to load
    :param sheet: name of the sheet in the workbook that is loaded
    :return: pandas dataFrame with imported data
    """
    import pandas as pd

    out = pd.read_excel(path, sheet_name=sheet)

    return out

In [6]:
# Specify path and sheet name in the prostate workbook
#filePath = 'C:/Users/PeterBakke/Documents/git/ML_fall2018/Data/Prostate.xlsx'
filePath = 'C:/Users/Greta/Documents/Github/ML_fall2018/Data/Prostate.xlsx'
#filePath = 'C:/Users/narisa/Documents/GitHub/ML_fall2018/Data/Prostate.xlsx'
sheet = 'Sheet1'

In [7]:
# load prostate data into dataFrame
myData = DataLoader(path=filePath, sheet=sheet)

# delete irrelevant columns
del myData['ID']
del myData['train']

In [8]:
# extract class names and encode with integers (dict)

classLabels = myData['Gleason'].values.tolist()
classNames = sorted(set(classLabels))
classDict = dict(zip(classNames, range(4)))

#del myData['Gleason']

attributeNames = list(myData.columns.values)

print(attributeNames)
print(classDict)

['lCaVol', 'lWeight', 'Age', 'lBPH', 'SVI', 'lCP', 'Gleason', 'pgg45', 'lPSA']
{6: 0, 7: 1, 8: 2, 9: 3}


In [9]:
# Extract vector y, convert to NumPy array
y = np.asarray([classDict[value] for value in classLabels])

# Convert dataFrame to numpy array
X = myData.values

# Compute values of N, M and C
N = len(y)
M = len(attributeNames)
C = len(classNames)

In [10]:
# Normalize data with mean and std
Y = (X - np.ones((N,1))*X.mean(axis=0)) / X.std(axis=0)

## Classification

In [11]:
# Remove attribute 5 (SVI) from X
X_classification = X[:,[0,1,2,3,5,6,7,8]]
print(X)
# Use attribute 5 (SVI) as y
y_classification = X[:,4]

[[-5.79818495e-01  2.76945900e+00  5.00000000e+01 -1.38629436e+00
   0.00000000e+00 -1.38629436e+00  6.00000000e+00  0.00000000e+00
  -4.30782900e-01]
 [-9.94252273e-01  3.31962600e+00  5.80000000e+01 -1.38629436e+00
   0.00000000e+00 -1.38629436e+00  6.00000000e+00  0.00000000e+00
  -1.62518900e-01]
 [-5.10825624e-01  2.69124300e+00  7.40000000e+01 -1.38629436e+00
   0.00000000e+00 -1.38629436e+00  7.00000000e+00  2.00000000e+01
  -1.62518900e-01]
 [-1.20397280e+00  3.28278900e+00  5.80000000e+01 -1.38629436e+00
   0.00000000e+00 -1.38629436e+00  6.00000000e+00  0.00000000e+00
  -1.62518900e-01]
 [ 7.51416089e-01  3.43237300e+00  6.20000000e+01 -1.38629436e+00
   0.00000000e+00 -1.38629436e+00  6.00000000e+00  0.00000000e+00
   3.71563600e-01]
 [-1.04982212e+00  3.22882600e+00  5.00000000e+01 -1.38629436e+00
   0.00000000e+00 -1.38629436e+00  6.00000000e+00  0.00000000e+00
   7.65467800e-01]
 [ 7.37164066e-01  3.47351800e+00  6.40000000e+01  6.15185640e-01
   0.00000000e+00 -1.3862943

### Logistic Regression

In [12]:
# Fit logistic regression model
model = lm.logistic.LogisticRegression()
model = model.fit(X_classification,y_classification)

# Classify observation according to SVI (0/1) and assess probabilities
y_est = model.predict(X_classification)
y_est_no_SVI_prob = model.predict_proba(X_classification)[:, 0] 

# Define a new data object (new type of wine), as in exercise 5.1.7
#x = np.array([6.9, 1.09, .06, 2.1, .0061, 12, 31, .99, 3.5, .44, 12]).reshape(1,-1)
# Evaluate the probability of x being a white wine (class=0) 
#x_class = model.predict_proba(x)[0,0]

# Evaluate classifier's misclassification rate over entire training data
misclass_rate = sum(np.abs(y_est - y_classification)) / float(len(y_est))

# Display classification results
#print('\nProbability of given sample not having a white wine: {0:.4f}'.format(x_class))
#print('\nOverall misclassification rate: {0:.3f}'.format(misclass_rate))

f = figure();
class0_ids = np.nonzero(y==0)[0].tolist()
plot(class0_ids, y_est_white_prob[class0_ids], '.y')
class1_ids = np.nonzero(y==1)[0].tolist()
plot(class1_ids, y_est_white_prob[class1_ids], '.r')
xlabel('Data object (men)'); ylabel('Predicted prob. of no seminal vesicle invation');
legend(['No seminal vesicle invasion', 'Seminal vesicle incation'])
ylim(-0.01,1.5)

NameError: name 'y_est_white_prob' is not defined

<Figure size 432x288 with 0 Axes>

### Two level cross validation for KNN

In [15]:
## Crossvalidation for KNN
# Create crossvalidation partition for evaluation
K_outer = 5
K_inner = 5
CV_outer = model_selection.KFold(n_splits=K_outer,shuffle=True)
#CV = model_selection.StratifiedKFold(n_splits=K)

# Initialize variables
Error_logreg = np.empty((K_outer,1))
Error_KNN_inner = np.empty((K_inner,3))
Error_NaïveB = np.empty((K_outer,1))
n_tested=0

K_KNN = [1, 10, 20]

k=0
classifier_lst = []

for train_outer_index, test_outer_index in CV_outer.split(X_classification,y_classification):
    print('CV-fold {0} of {1}'.format(k+1,K_outer))
    
    # extract training and test set for current CV fold
    X_train_outer = X_classification[train_outer_index,:]
    y_train_outer = y_classification[train_outer_index]
    X_test_outer = X_classification[test_outer_index,:]
    y_test_outer = y_classification[test_outer_index]
    
    CV_inner = model_selection.KFold(n_splits=K_inner,shuffle=True)
    
    kk=0
    for train_inner_index, test_inner_index in CV_inner.split(X_train_outer,y_train_outer):
        print('CV-fold {0} of {1}'.format(k+1,K_inner))

        # extract training and test set for current CV fold
        X_train_inner = X_train_outer[train_inner_index,:]
        y_train_inner = y_train_outer[train_inner_index]
        X_test_inner = X_train_outer[test_inner_index,:]
        y_test_inner = y_train_outer[test_inner_index]
        
        print(len(X_train_inner))
        print(len(y_train_inner))
        
        for i, KNN in enumerate(K_KNN):
        
            # Distance metric (corresponds to 2nd norm, euclidean distance).
            # You can set dist=1 to obtain manhattan distance (cityblock distance).
            dist=1
    
            knclassifier = KNeighborsClassifier(n_neighbors=K_KNN, p=dist);
            knclassifier.fit(X_train_inner, y_train_inner);
            classifier_lst.append(knclassifier)
            
            y_KNN = knclassifier.predict(X_test_inner);
            Error_KNN_inner[kk] = 100*(y_KNN!=y_test_inner).sum().astype(float)/len(y_test_inner)
            kk+=1
            
            
            # Take model out here with optimal K
        # Test min error 
        index_min = np.argmin(Error_KNN_inner)
        
        print('Inner error is (with index)')
        print(Error_KNN_inner)
        print(index_min)
    # Test here
    
    optimal_model = classifier_lst[index_min]
    
    
        
    
    
    
    

CV-fold 1 of 5
CV-fold 1 of 5
61
61


TypeError: '<' not supported between instances of 'list' and 'int'

In [ ]:
# Complexity parameters
K_KNN = [1, 5, 10]

# K-fold crossvalidation
K = 10
CV = model_selection.KFold(n_splits=K,shuffle=True)

# Initialize variable
Error_train = np.empty((len(K_KNN),K))
Error_test = np.empty((len(K_KNN),K))

k=0
for train_index, test_index in CV.split(X):
    print('Computing CV fold: {0}/{1}..'.format(k+1,K))

    # extract training and test set for current CV fold
    X_train, y_train = X[train_index,:], y[train_index]
    X_test, y_test = X[test_index,:], y[test_index]

    for i, t in enumerate(tc):
        # Fit decision tree classifier, Gini split criterion, different pruning levels
        dtc = tree.DecisionTreeClassifier(criterion='gini', max_depth=t)
        dtc = dtc.fit(X_train,y_train.ravel())
        y_est_test = dtc.predict(X_test)
        y_est_train = dtc.predict(X_train)
        # Evaluate misclassification rate over train/test data (in this CV fold)
        misclass_rate_test = sum(np.abs(y_est_test - y_test)) / float(len(y_est_test))
        misclass_rate_train = sum(np.abs(y_est_train - y_train)) / float(len(y_est_train))
        Error_test[i,k], Error_train[i,k] = misclass_rate_test, misclass_rate_train
    k+=1

In [59]:
from sklearn.model_selection import cross_validate, cross_val_score 
## Crossvalidation for KNN
# Create crossvalidation partition for evaluation
K_outer = 2
CV_outer = model_selection.KFold(n_splits=K_outer,shuffle=True)
#CV = model_selection.StratifiedKFold(n_splits=K)

# Initialize variables
#Error_logreg = np.empty((K,1))
#Error_KNN_inner = np.empty((K,1))
#Error_NaïveB = np.empty((K,1))
#n_tested=0



k=0
for train_outer_index, test_outer_index in CV_outer.split(X_classification,y_classification):
    print('CV-fold {0} of {1}'.format(k+1,K_outer))
    
    # extract training and test set for current CV fold
    X_train_outer = X_classification[train_outer_index,:]
    y_train_outer = y_classification[train_outer_index]
    X_test_outer = X_classification[test_outer_index,:]
    y_test_outer = y_classification[test_outer_index]
 
    # Distance metric (corresponds to 2nd norm, euclidean distance).
    # You can set dist=1 to obtain manhattan distance (cityblock distance).
    dist= 2
    cv_scores = []
    for K_KNN in [1, 5, 10]:
        knclassifier = KNeighborsClassifier(n_neighbors=K_KNN, p=dist); 
        #cv_results = cross_validate(knclassifier, X_train_outer, y_train_outer, cv=2, return_train_score=False)
        #print(sorted(cv_results.keys()))
        #print(cv_results['test_score'])
        scores = cross_val_score(knclassifier, X_train_outer, y_train_outer, cv=2, scoring='accuracy')
        #cv_scores.append(scores.mean())
    print(scores)
    k+=1
    
            


CV-fold 1 of 2
[0.75  0.625]
CV-fold 2 of 2
[0.8        0.83333333]


In [57]:
help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs')
    Evaluate a score by cross-validation
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like
        The data to fit. Can be for example a list, or an array.
    
    y : array-like, optional, default: None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like, with shape (n_samples,), optional
        Group labels for the samples used while splitting the dataset into
        train/test set.
    
    scoring : string, callable or None, optional, default: None
        A string (see model evaluation documentation) or
        a scorer callable obje